In [1]:
import pickle

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from tensorflow import keras
import kerasncp as kncp

import os
from typing import Iterable, Dict
import tensorflow as tf
import kerasncp as kncp
from kerasncp.tf import LTCCell, WiredCfcCell
from tensorflow import keras
import numpy as np
from matplotlib.image import imread
from tqdm import tqdm
from PIL import Image
import pandas as pd
import time
from keras_models import generate_ncp_model
from train_test_loader import get_dataset_multi, get_val_dataset_multi

2024-12-12 10:46:00.912871: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
training_root = "../../fly_to_target_dataset/diff_coreset"
val_root = "../../fly_to_target_dataset/test_data"
DROPOUT = 0.1

DEFAULT_NCP_SEED = 22222

IMAGE_SHAPE = (144, 256, 3)
IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

In [3]:
def generate_augmentation_layers(x, augmentation_params: Dict, single_step: bool):
    # translate -> rotate -> zoom -> noise
    trans = augmentation_params.get('translation', None)
    rot = augmentation_params.get('rotation', None)
    zoom = augmentation_params.get('zoom', None)
    noise = augmentation_params.get('noise', None)

    if trans is not None:
        x = wrap_time(keras.layers.experimental.preprocessing.RandomTranslation(
            height_factor=trans, width_factor=trans), single_step)(x)

    if rot is not None:
        x = wrap_time(keras.layers.experimental.preprocessing.RandomRotation(rot), single_step)(x)

    if zoom is not None:
        x = wrap_time(keras.layers.experimental.preprocessing.RandomZoom(
            height_factor=zoom, width_factor=zoom), single_step)(x)

    if noise:
        x = wrap_time(keras.layers.GaussianNoise(stddev=noise), single_step)(x)

    return x


def generate_normalization_layers(x, single_step: bool):
    rescaling_layer = keras.layers.experimental.preprocessing.Rescaling(1. / 255)

    normalization_layer = keras.layers.experimental.preprocessing.Normalization(
        mean=[0.6042006463205742, 0.6042006463205742, 0.6042006880578502],
        variance=[0.0103, 0.0103, 0.0103])

    x = rescaling_layer(x)
    x = wrap_time(normalization_layer, single_step)(x)
    return x


def wrap_time(layer, single_step: bool):
    """
    Helper function that wraps layer in a timedistributed or not depending on the arguments of this function
    """
    if not single_step:
        return keras.layers.TimeDistributed(layer)
    else:
        return layer


def generate_network_trunk(seq_len,
                           image_shape,
                           augmentation_params: Dict = None,
                           batch_size=None,
                           single_step: bool = False,
                           no_norm_layer: bool = False, ):
    """
    Generates CNN image processing backbone used in all recurrent models. Uses Keras.Functional API

    returns input to be used in Keras.Model and x, a tensor that represents the output of the network that has shape
    (batch [None], seq_len, num_units) if single step is false and (batch [None], num_units) if single step is true.
    Input has shape (batch, h, w, c) if single step is True and (batch, seq, h, w, c) otherwise

    """

    if single_step:
        inputs = keras.Input(shape=image_shape)
    else:
        inputs = keras.Input(batch_input_shape=(batch_size, seq_len, *image_shape))

    x = inputs

    if not no_norm_layer:
        x = generate_normalization_layers(x, single_step)

    if augmentation_params is not None:
        x = generate_augmentation_layers(x, augmentation_params, single_step)

    # Conv Layers
    x = wrap_time(keras.layers.Conv2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu'), single_step)(
        x)
    x = wrap_time(keras.layers.Conv2D(filters=36, kernel_size=(5, 5), strides=(2, 2), activation='relu'), single_step)(
        x)
    x = wrap_time(keras.layers.Conv2D(filters=48, kernel_size=(5, 5), strides=(2, 2), activation='relu'), single_step)(
        x)
    x = wrap_time(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu'), single_step)(
        x)
    x = wrap_time(keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2), activation='relu'), single_step)(
        x)

    # fully connected layers
    x = wrap_time(keras.layers.Flatten(), single_step)(x)
    x = wrap_time(keras.layers.Dense(units=128, activation='linear'), single_step)(x)
    x = wrap_time(keras.layers.Dropout(rate=DROPOUT), single_step)(x)

    return inputs, x

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp

tfd = tfp.distributions

DROPOUT = 0.3

class PPOModel(tf.keras.Model):
    def __init__(self, seq_len, image_shape, augmentation_params=None, batch_size=None, single_step=False, no_norm_layer=False):
        super(PPOModel, self).__init__()

        # Generate the CNN trunk
        self.inputs_image, self.cnn_features = generate_network_trunk(
            seq_len,
            image_shape,
            augmentation_params=augmentation_params,
            batch_size=batch_size,
            single_step=single_step,
            no_norm_layer=no_norm_layer,
        )

        # Actor Network (for velocity predictions)
        self.actor_dense1 = layers.Dense(64, activation='relu')
        self.actor_dense2 = layers.Dense(32, activation='relu')
        self.actor_output = layers.Dense(4, activation='tanh')  # Outputs [v_x, v_y, v_z, omega_z]

        # Critic Network (for value estimation)
        self.critic_dense1 = layers.Dense(64, activation='relu')
        self.critic_dense2 = layers.Dense(32, activation='relu')
        self.critic_output = layers.Dense(1, activation='linear')

    def call(self, inputs):
        x = self.cnn_features

        # Actor forward pass
        actor_hidden = self.actor_dense1(x)
        actor_hidden = self.actor_dense2(actor_hidden)
        actions = self.actor_output(actor_hidden)

        # Critic forward pass
        critic_hidden = self.critic_dense1(x)
        critic_hidden = self.critic_dense2(critic_hidden)
        value = self.critic_output(critic_hidden)

        return actions, value

def ppo_loss(old_probs, actions, advantages, clip_ratio=0.2):
    def loss_fn(y_true, y_pred):
        new_probs = tfd.MultivariateNormalDiag(loc=y_pred).log_prob(actions)
        ratio = tf.exp(new_probs - old_probs)
        clipped_ratio = tf.clip_by_value(ratio, 1 - clip_ratio, 1 + clip_ratio)
        loss_actor = -tf.reduce_mean(tf.minimum(ratio * advantages, clipped_ratio * advantages))

        # Critic loss (value estimation)
        y_true_value, y_pred_value = y_true[:, -1], y_pred[:, -1]
        loss_critic = keras.losses.MSE(y_true_value, y_pred_value)

        return loss_actor + 0.5 * loss_critic

    return loss_fn

def build_ppo_model(seq_len, image_shape, augmentation_params=None, batch_size=None, single_step=False, no_norm_layer=False):
    # Initialize the model
    model = PPOModel(seq_len, image_shape, augmentation_params, batch_size, single_step, no_norm_layer)

    # Compile the model with PPO loss
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4))

    return model


In [11]:
!export TF_CPP_MIN_LOG_LEVEL=2

In [12]:
shift: int = 1
stride: int = 1
# decay_rate: float = 0.95
val_split: float = 0.2
label_scale: float = 1
seq_len = 64
val_split: float = 0.1
label_scale: float = 1

training_dataset = get_dataset_multi(training_root, IMAGE_SHAPE, seq_len, shift, stride, val_split, label_scale, extra_data_root=None)
print('load dataset shape', training_dataset.element_spec)
training_dataset = training_dataset.batch(64)

labels [[-5.45208991e-01  5.56323528e-01 -9.62745726e-01  5.51216781e-01]
 [-4.69735712e-01  5.59266984e-01 -8.87602270e-01  5.16884804e-01]
 [-3.96264791e-01  5.48152566e-01 -8.28002930e-01  4.87919092e-01]
 [-3.18426251e-01  5.24390161e-01 -7.62720525e-01  4.55274433e-01]
 [-2.60237217e-01  4.96815681e-01 -7.08253622e-01  4.27806526e-01]
 [-2.19786286e-01  4.72018570e-01 -6.70199931e-01  4.07966048e-01]
 [-1.71101779e-01  4.34729010e-01 -6.20564520e-01  3.81809920e-01]
 [-1.26528889e-01  3.93339336e-01 -5.77598214e-01  3.56382489e-01]
 [-9.09503549e-02  3.50732327e-01 -5.36118031e-01  3.34085882e-01]
 [-7.03007281e-02  3.23684692e-01 -5.10422289e-01  3.17597598e-01]
 [-4.33358178e-02  2.80392528e-01 -4.69446898e-01  2.94971615e-01]
 [-2.49505118e-02  2.43477389e-01 -4.32806820e-01  2.73863882e-01]
 [-1.16705829e-02  2.13092744e-01 -4.04083222e-01  2.56695092e-01]
 [-6.24483393e-04  1.82767749e-01 -3.74043435e-01  2.38524824e-01]
 [ 4.71147848e-03  1.65640324e-01 -3.56202573e-01  2.27

In [13]:
batch_size = None
seq_len = 64
augmentation_params = None
single_step = False
no_norm_layer = False


model = build_ppo_model(seq_len, IMAGE_SHAPE, augmentation_params, batch_size, single_step, no_norm_layer)

epochs = 10
history = model.fit(x=training_dataset, epochs=epochs,verbose=1, use_multiprocessing=False, workers=1, max_queue_size=5)
print(history)

Epoch 1/10


ValueError: in user code:

    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:788 run_step  **
        outputs = model.t